***

# Data Cleaning

  > [Zachary Kilhoffer](https://zkilhoffer.github.io/ "Author's website")

  > Updated 2024-06-17

***

## Description
- The code is part of the pipelines described in: 
  - Kilhoffer, Z. et al. (2024 in press). "Cloud Privacy Beyond Legal Compliance: An NLP analysis of certifiable privacy and security standards".
- The paper will be released on 2024-06-28 at the [IEEE Cloud Summit](https://www.ieeecloudsummit.org/2024-program "Program of IEEE Cloud Summit 2024").
- This script cleans/cleanses the text data before subsequent work, notably creating embeddings.
  - By making calls to OpenAI, we use an LLM to batch correct errors that remain from the optical character recognition (OCR) process

### Input files:
 `df = pd.read_excel('df_2023-12-16.xlsx')`

 `reference_only_df = pd.read_excel('reference_only_df_2023-12-16.xlsx')`

### Output files:
`df_2023-12-16.xlsx`

`reference_only_df_2023-12-16.xlsx`

***

In [1]:
import warnings
import pandas as pd
import numpy as np
import openpyxl

from openai import OpenAI
import nltk
from nltk.corpus import words

# Download the words dataset from nltk
nltk.download('words')
nltk.download('punkt')

/Users/Zak/Library/CloudStorage/Dropbox/Documents/Education/PhD/Cisco/Cloud Certifications Evaluation V2/Privacy Certifications 2023-12-12/ciscovenv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package words to /Users/Zak/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Zak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Setup

In [2]:
# display tweaks
pd.set_option("display.max_colwidth", 200)  # how much text is showing within a cell
pd.set_option("display.max_columns", False)
pd.set_option("display.max_rows", False)
warnings.filterwarnings("ignore")

In [3]:
# load data, which is all OCR'd text
df = pd.read_excel(
    r"/Users/Zak/Library/CloudStorage/Dropbox/Documents/Education/PhD/Cisco/Cloud Certifications Evaluation V2/Privacy Certifications 2023-12-12/data/intermediate/all_document_data.xlsx",
    index_col=0,
)

#### TODO: control_text not corrected here

In [4]:
df.head(1)

,control_category,control_code,control_name,control_text,page,document
0.0,Organisation of Information Security (OIS),OIS-01,Information Security Management System (ISMS),Basic Criterion The Cloud Service Provider operates an information security management system (ISMS) in accordance with ISO/IEC 27001. The scope of the ISMS covers the Cloud Service Provider’s or...,36,c5


In [5]:
# remove rows where control_text is np.Nan
print(f'Number of np.NaN in df["control_text"]: {df["control_text"].value_counts(dropna=False)[0]}.')
df.dropna(subset=['control_text'], inplace=True)

Number of np.NaN in df["control_text"]: 106.


# Fix OCR Errors

Notes
- c5: control_name is very short, control_text is detail
- ccm
- eu_coc: control_name is closer to control text. control_text is actually "control guidance"
- fedramp
- iso_27001
- iso_27002 - most of the control_text is referential, refering to iso/iec 27002: 2013
- iso_27017
- iso_27018
- iso_27701
- nist
- soc2

In [6]:
# lowercase text
df['control_text'] = df['control_text'].apply(lambda x: x.lower())
df['control_name'] = df['control_name'].apply(lambda x: x.lower() if isinstance(x, str) else '')
df['control_category'] = df['control_category'].apply(lambda x: x.lower() if isinstance(x, str) else '')

In [7]:
# after visually inspecting the possibly wrong column, made this replacement list
replacements = {'crite-ria': 'criteria',
                '1so/iec': 'iso/iec',
                '1so': 'iso',
                '(<>)': '',
                'iso/ iec': 'iso/iec',
                'applies.implementation': 'applies. implementation',
                'ofiso/iec': 'of iso/iec',
                'iso/lec': 'iso/iec',
                ' ofthe ': ' of the ',
                ' pll': ' pii',
                'cus-tomer': 'customer',
                '1s0/iec': 'iso/iec',
                'iso/i ec': 'iso/iec',
                'per-sonal': 'personal',
                'infor-mation': 'information',
                'applies.public': 'applies. public',
                'thecustomer': 'the customer',
                'incidentresponse': 'incident response',
                'integritychecking': 'integrity checking',
                'incident-response': 'incident response',
                'integrity-checking': 'integrity checking',
                'service-cloud': 'service cloud',
                'servicecloud': 'service cloud',
                'integrity-checking': 'integrity checking' ,
                'provider-cloud': 'provider cloud',
                'providercloud': 'provider cloud',
                'organization-level': 'organization level',
                'csp': 'cloud service provider',
                'basic criterion ': 'basic criterion: ',
                'inter- national': 'international',
                'organiza- tions': 'organizations',
                'ad-dress': 'address',
                'regard-ing': 'regarding'
                }
# Replace the strings in 'control_text' column
for key, value in replacements.items():
    df['control_text'] = df['control_text'].str.replace(key, value)

# Replace the strings in 'control_name' column
for key, value in replacements.items():
    df['control_name'] = df['control_name'].str.replace(key, value)

In [15]:
# replace /n
df['control_text'] = df['control_text'].str.replace('\n', ' ')

# remove extra spaces
df["control_text"] = df["control_text"].apply(lambda x: " ".join(x.split()))

# remove trailing/leading spaces
df["control_text"] = df["control_text"].apply(lambda x: x.strip() if type(x) == str else x)
df["control_name"] = df["control_name"].dropna().apply(lambda x: x.strip() if type(x) == str else x)
df["control_code"] = df["control_code"].dropna().apply(lambda x: x.strip() if type(x) == str else x)
df["control_category"] = df["control_category"].dropna().apply(lambda x: x.strip() if type(x) == str else x)

# could be that PLMs misunderstand the acronym 'pii', so replace with whole phrase
df['control_text'] = df['control_text'].str.replace('pii', 'personally identifiable information')
df['control_name'] = df['control_name'].str.replace('pii', 'personally identifiable information')

In [ ]:
# # Find candidates for misspellings to manually check, build dictionary of string replacements

# # Load the list of English words
# english_words = words.words()

# # Find words not in English
# def non_english_words(text):
#     words = text.split()
#     return ' '.join([word for word in words if word not in english_words])

# df['possibly_wrong'] = df['control_text'].apply(non_english_words)
# df.drop(axis=1, columns=['possibly_wrong'], inplace=True)  # cleanup

In [16]:
# dict of string replacements for control texts, made by visually inspecting the possibly wrong column
replacements = {'crite-ria': 'criteria',
                '1so/iec': 'iso/iec',
                '1so': 'iso',
                '(<>)': '',
                'iso/ iec': 'iso/iec',
                'applies.implementation': 'applies. implementation',
                'ofiso/iec': 'of iso/iec',
                'iso/lec': 'iso/iec',
                ' ofthe ': ' of the ',
                ' pll': ' pii',
                'cus-tomer': 'customer',
                '1s0/iec': 'iso/iec',
                'iso/i ec': 'iso/iec',
                'per-sonal': 'personal',
                'infor-mation': 'information',
                'applies.public': 'applies. public',
                'thecustomer': 'the customer',
                'incidentresponse': 'incident response',
                'integritychecking': 'integrity checking',
                'incident-response': 'incident response',
                'integrity-checking': 'integrity checking',
                'service-cloud': 'service cloud',
                'servicecloud': 'service cloud',
                'integrity-checking': 'integrity checking' ,
                'provider-cloud': 'provider cloud',
                'providercloud': 'provider cloud',
                'organization-level': 'organization level',
                'csp': 'cloud service provider',
                'basic criterion ': 'basic criterion: ',
                'inter- national': 'international',
                'organiza- tions': 'organizations',
                'ad-dress': 'address',
                'regard-ing': 'regarding'
                }

# Replace the strings in 'control_text' column
for key, value in replacements.items():
    df['control_text'] = df['control_text'].str.replace(key, value)

# Replace the strings in 'control_name' column
for key, value in replacements.items():
    df['control_name'] = df['control_name'].str.replace(key, value)

## OpenAI OCR error correction

In [17]:
# function to retrieve key
def read_key_from_file(filename=r"/Users/Zak/Library/CloudStorage/Dropbox/Documents/Education/PhD/Cisco/Cloud Certifications Evaluation V2/Privacy Certifications 2023-12-12/key.txt"):
    with open(filename, 'r') as file:
        return file.read().strip()

# initialize client
client = OpenAI(organization='org-NZYm3UxLmOOtDWjZAqHm6X9i', api_key=read_key_from_file())

In [18]:
# text we'll check
text_to_check = df.loc[1506]['control_text']
text_to_check

'control the organization should determine and securely maintain the necessary records in support of its obligations for the processing of personally identifiable information. implementation guidance a way to maintain records of the processing of personally identifiable information is to have an inventory or list of the personally identifiable information processing activities that theorganization performs. such an inventory can include: the type of processing; the purposes for the processing; a description of the categories of personally identifiable information and personally identifiable information principals (e.g. children); the categories of recipients to whom personally identifiable information has been or will be disclosed, including recipients in third countries or international organizations; a general description of the technical and organizational security measures; and a privacy impact assessment report. such an inventory should have an owner who is responsible for its acc

In [19]:
# querying API
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "I will provide you text from cybersecurity and privacy documentation like ISO-IEC 27017, FedRamp, etc. The text is the result of imperfect optical character recognition (OCR), which needs to have errors fixed. Your job is to return the same text with OCR errors corrected. Some text I give you will include headings. This is especially likely if there's no ending punctuation. When some part of the text is likely to be a heading, treat it like its own sentence, separating it from other text with a period. Examples of headings may include something like 'Controls' or 'Responsibilities of data controller'."},
    {"role": "user", "content": text_to_check}
  ]
)

# completion.choices[0].message['content']
completion.choices[0].message.content

'Control: The organization should determine and securely maintain the necessary records in support of its obligations for the processing of personally identifiable information. \n\nImplementation guidance: A way to maintain records of the processing of personally identifiable information is to have an inventory or list of the personally identifiable information processing activities that the organization performs. \n\nSuch an inventory can include:\n\n- The type of processing\n- The purposes for the processing\n- A description of the categories of personally identifiable information and personally identifiable information principals (e.g., children)\n- The categories of recipients to whom personally identifiable information has been or will be disclosed, including recipients in third countries or international organizations\n- A general description of the technical and organizational security measures\n- A privacy impact assessment report\n\nSuch an inventory should have an owner who i

In [20]:
# function to correct OCR errored text
def ocr_correction(text_to_check):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "I will provide you text from cybersecurity and privacy documentation like FedRamp, SOC-2, etc. The text is the result of imperfect optical character recognition (OCR), which needs to have errors fixed. Your job is to return the same text with OCR errors corrected. Some text I give you will include headings. Text is especially likely to be a heading if there's no ending punctuation. When some part of the text is likely to be a heading, treat it like its own sentence, separating it from other text with a period. Examples of headings may include something like 'Controls' or 'Responsibilities of data controller'.",
            },
            {"role": "user", "content": text_to_check},
        ],
    )
    return completion.choices[0].message.content

In [ ]:
Note: correcting the text of the control's name was not successful. Once in a while ChatGPT just started writing stuff about the name of the control, rather than simply returning a corrected version of it.

In [24]:
# applying function to correct OCR text
df['control_text_corrected'] = df['control_text'].apply(lambda x: ocr_correction(x))

# Create full_control_text column

In [32]:
# combining all potentially useful information for control text
# goal is to separate the control category, name, and text with a period
# this requires a conditional so we don't have full_control_texts that start with '.' sometimes

def concatenate_with_condition(row):
    result = f"{row['control_category']}".capitalize()
    
    if len(result) > 1:
        result += '. '
    if len(row['control_name']) > 2:
        result += f"{row['control_name'].strip().capitalize()}. "
    result += f"{row['control_text_corrected']}"     

    return result

In [33]:
# Apply the custom function to create the new column
df['full_control_text'] = df.apply(concatenate_with_condition, axis=1)

print(df.shape)

(1661, 9)


In [34]:
# fixing the "nan. <control text>" problem for some documents. the documents: iso_27002, iso_27018, iso_27017, eu_coc, iso_27701
df['full_control_text'] = df['full_control_text'].str.replace('Nan. ', '')

# Identifying reference-only controls

- It's valuable to remove controls that are mere references like "the blah blah in ISO/IEC 10.1.2 applies"
- But some of the longer texts have interesting information, so we keep them (>250 characters)
- We use rules and regex to identify the reference controls

In [35]:
df['document'].value_counts(dropna=False)

document
fedramp      410
iso_27701    238
ccm          197
iso_27017    154
c5           121
iso_27018    114
iso_27002    110
nist         100
iso_27001     93
eu_coc        62
soc2          61
NaN            1
Name: count, dtype: int64

In [36]:
# Define the updated regex patterns
pattern1 = r"the control.*implementation guidance and other information stated in.*appl"
pattern2 = r"the requirements stated in.*appl"
pattern3 = r"the requirement stated in.*appl"
pattern4 = r"the objective specified in.*appl"
pattern5 = r"the objectives specified in.*appl"
pattern6 = r"control.*and the associated implementation guidance.*specified in.*appl"
pattern7 = r"the objective specified in.*iso.*appl"
pattern8 = r"the objectives specified in iso.*appl"
pattern9 = r"control.*implementation guidance and other information.*appl"

reference_only_df = df[
    (df["control_text"].str.len() < 250)
    & (
        df["control_text"].str.contains(pattern1, regex=True)
        | df["control_text"].str.contains(pattern2, regex=True)
        | df["control_text"].str.contains(pattern3, regex=True)
        | df["control_text"].str.contains(pattern4, regex=True)
        | df["control_text"].str.contains(pattern5, regex=True)
        | df["control_text"].str.contains(pattern6, regex=True)
        | df["control_text"].str.contains(pattern7, regex=True)
        | df["control_text"].str.contains(pattern8, regex=True)
        | df["control_text"].str.contains(pattern9, regex=True)
    )
]

# drop NIST (not certifiable)
df.drop(df[df['document'] == 'nist'].index, inplace=True)

# remove ISO/IEC 27001
df_27001 = df[df['document'] == 'iso_27001']
reference_only_df = pd.concat([df_27001, reference_only_df])

# filter the reference_only_df out of df
df.drop(reference_only_df.index, inplace=True)

# Saving results

In [40]:
# inspect reference only df
reference_only_df.head()

,control_category,control_code,control_name,control_text,page,document,control_text_corrected,control_name_corrected,full_control_text
792.0,organizational controls,5.1,policies for information security,"control information security policy and topic-specific policies shall be defined, approved by management, published, communicated to and acknowledged by relevant personnel and relevant interested ...",16,iso_27001,"Control information security policy and topic-specific policies shall be defined, approved by management, published, communicated to, and acknowledged by relevant personnel and relevant interested...",Policies for Information Security.,"Organizational controls. Policies for information security. Control information security policy and topic-specific policies shall be defined, approved by management, published, communicated to, an..."
793.0,organizational controls,5.2,information security roles and responsibilities,control information security roles and responsibilities shall be defined and allocated according to the organization needs.,16,iso_27001,Control: Information Security Roles and Responsibilities\n\nShall be defined and allocated according to the organization's needs.,Information Security Roles and Responsibilities.,Organizational controls. Information security roles and responsibilities. Control: Information Security Roles and Responsibilities\n\nShall be defined and allocated according to the organization's...
794.0,organizational controls,5.3,segregation of duties,control conflicting duties and conflicting areas of responsibility shall be segregated.,16,iso_27001,Control conflicting duties and conflicting areas of responsibility shall be segregated.,Segregation of Duties.,Organizational controls. Segregation of duties. Control conflicting duties and conflicting areas of responsibility shall be segregated.
795.0,organizational controls,5.4,management responsibilities,"control management shall require all personnel to apply information security in accordance with the established information security policy, topic-specific policies and procedures of the organizat...",16,iso_27001,"Control management shall require all personnel to apply information security in accordance with the established information security policy, topic-specific policies, and procedures of the organiza...",Management Responsibilities.,"Organizational controls. Management responsibilities. Control management shall require all personnel to apply information security in accordance with the established information security policy, t..."
796.0,organizational controls,5.5,contact with authorities,control the organization shall establish and maintain contact with relevant authorities.,16,iso_27001,Control: The organization shall establish and maintain contact with relevant authorities.,Contact with Authorities.,Organizational controls. Contact with authorities. Control: The organization shall establish and maintain contact with relevant authorities.


In [39]:
# inspect main df
df.head()

,control_category,control_code,control_name,control_text,page,document,control_text_corrected,full_control_text
0.0,organisation of information security (ois),OIS-01,information security management system (isms),basic criterion: the cloud service provider operates an information security management system (isms) in accordance with iso/iec 27001. the scope of the isms covers the cloud service provider’s or...,36,c5,Basic criterion: The cloud service provider operates an Information Security Management System (ISMS) in accordance with ISO/IEC 27001. The scope of the ISMS covers the cloud service provider's or...,Organisation of information security (ois). Information security management system (isms). Basic criterion: The cloud service provider operates an Information Security Management System (ISMS) in ...
1.0,organisation of information security (ois),OIS-02,information security policy,basic criterion: the top management of the cloud service provider has adopted an information security policy and communicated it to internal and external employees as well as cloud customers. the ...,36,c5,"Basic criterion: The top management of the cloud service provider has adopted an information security policy and communicated it to internal and external employees, as well as cloud customers. The...",Organisation of information security (ois). Information security policy. Basic criterion: The top management of the cloud service provider has adopted an information security policy and communicat...
2.0,organisation of information security (ois),OIS-03,interfaces and dependencies,basic criterion: interfaces and dependencies between cloud service delivery activities performed by the cloud service provider and activities performed by third parties are documented and communic...,37,c5,Basic criterion: Interfaces and dependencies between cloud service delivery activities performed by the cloud service provider and activities performed by third parties are documented and communic...,Organisation of information security (ois). Interfaces and dependencies. Basic criterion: Interfaces and dependencies between cloud service delivery activities performed by the cloud service provi...
3.0,organisation of information security (ois),OIS-04,segregation of duties,basic criterion: conflicting tasks and responsibilities are separated based on an ois-06 risk assessment to reduce the risk of unauthorised or unintended changes or misuse of cloud customer data p...,38,c5,Basic criterion: Conflicting tasks and responsibilities are separated based on an OIS-06 risk assessment to reduce the risk of unauthorized or unintended changes or misuse of cloud customer data p...,Organisation of information security (ois). Segregation of duties. Basic criterion: Conflicting tasks and responsibilities are separated based on an OIS-06 risk assessment to reduce the risk of un...
4.0,organisation of information security (ois),OIS-05,contact with relevant government agencies and interest groups,basic criterion: the cloud service provider leverages relevant authorities and interest groups in order to stay informed about current threats and vulnerabilities. the information flows into the p...,38,c5,Basic criterion: The cloud service provider leverages relevant authorities and interest groups in order to stay informed about current threats and vulnerabilities. The information flows into the p...,Organisation of information security (ois). Contact with relevant government agencies and interest groups. Basic criterion: The cloud service provider leverages relevant authorities and interest g...


In [41]:
# further inspection
reference_only_df['document'].value_counts()

document
iso_27017    107
iso_27701     97
iso_27001     93
iso_27018     70
iso_27002     69
Name: count, dtype: int64

In [42]:
# further inspection
df['document'].value_counts()

document
fedramp      410
ccm          197
iso_27701    141
c5           121
eu_coc        62
soc2          61
iso_27017     47
iso_27018     44
iso_27002     41
Name: count, dtype: int64

In [43]:
# savings results
df.to_excel('df_2023-12-16.xlsx')
reference_only_df.to_excel('reference_only_df_2023-12-16.xlsx')